In [1]:
import os
import pandas as pd

In [2]:
my_dir = "D:/Thesis/Enveloppes/input_data/contacts_min/"

## Fonction pour construire les enveloppes

In [3]:
def identifier_enveloppe(nom, seuil, cutt_off):
    
    res = []
    for file_name in os.listdir(my_dir):

        df = pd.read_excel(os.path.join(my_dir, file_name))

        dfs = []
        for unique_val in df['Cap2'].unique():
            df_filtered = df[(df["Cap2"] == unique_val)]

            groupes = []
            groupe_actuel = []

            for i in range(len(df_filtered) - 1):
                current_row = df_filtered.iloc[i]
                next_row = df_filtered.iloc[i + 1]

                if not groupe_actuel:
                    groupe_actuel.append((current_row['time_min'], current_row['mean_dist']))
                else:
                    # Vérification de la proximité de la distance et de la consécutivité du temps
                    if abs(groupe_actuel[-1][1] - current_row['mean_dist']) <= seuil and next_row['time_min'] - current_row['time_min'] == 1:
                        groupe_actuel.append((current_row['time_min'], current_row['mean_dist']))
                    else:
                        groupes.append(groupe_actuel)
                        groupe_actuel = [(current_row['time_min'], current_row['mean_dist'])]

            if groupe_actuel:
                groupes.append(groupe_actuel)

            # Filtrer les groupes selon la condition de distance
            groupes_filtrés = [groupe for groupe in groupes if all(d <= cutt_off for _, d in groupe)]
            top3_groupes = sorted(groupes_filtrés, key=len, reverse=True)[:3]

            data = []
            intervals = []        


            if (len(top3_groupes)> 0):

                # Attribuer un ID unique à chaque groupe du top 3
                for group_id, groupe in enumerate(top3_groupes, start=1):

                    start_time = groupe[0][0]
                    end_time = groupe[-1][0]
                    intervals.append({'Groupe_ID': group_id, 'Debut': start_time, 'Fin': end_time})
                    for point in groupe:
                            # Pour chaque observation dans le groupe, nous stockons l'ID du groupe, le temps et la distance
                            data.append({'Groupe_ID': group_id, 'Temps': point[0], 'Distance': point[1]})         

                # Convertir les données et les intervalles en DataFrames
                df_top3 = pd.DataFrame(data)
                df_intervals = pd.DataFrame(intervals)

                # Calculer la durée et la médiane pour chaque groupe
                df_intervals['Duree_min'] = df_intervals['Fin'] - df_intervals['Debut'] + 1
                #df_intervals['Duree_min'] = df_intervals['Duree_sec'] / 60
                df_intervals['Distance Mediane'] = df_intervals['Groupe_ID'].apply(lambda x: df_top3[df_top3['Groupe_ID'] == x]['Distance'].median())

                # fusionner les dataframes
                df_merged = pd.merge(df_top3, df_intervals, on="Groupe_ID")

                summary = df_merged[["Groupe_ID", "Duree_min", "Distance Mediane"]].drop_duplicates()

                new_df = pd.DataFrame()

                
                for index, row in summary.iterrows():

                    # Créer les nouveaux noms de colonnes
                    col1 = f"Duree_min_{row['Groupe_ID']}"
                    col2 = f"Distance Mediane_{row['Groupe_ID']}"

                    # Insérer les données dans les colonnes du nouveau DataFrame 
                    new_df.at[0, col1] = row['Duree_min']  
                    new_df.at[0, col2] = row['Distance Mediane']

                new_df.insert(0, 'cap_ID', unique_val)

                dfs.append(new_df)

            else:
                continue

            temp_df = pd.concat(dfs, ignore_index=True)
    

        nom_base = file_name.rsplit('.', 1)[0]

        partie_voulue = nom_base.rsplit('_', 2)[0]

        temp_df.insert(0, 'experiment', partie_voulue)

        res.append(temp_df)

    final_result = pd.concat(res, ignore_index=True)
    
    nom_fichier = f'{nom}_{seuil}.xlsx'
    
    if (cutt_off == 1):
        chemin = "D:/Thesis/Enveloppes/output_data/By_minutes/under_1m/"
    elif (cutt_off == 2):
        chemin = "D:/Thesis/Enveloppes/output_data/By_minutes/under_2m/"
    
    return final_result.to_excel(os.path.join(chemin, nom_fichier))


## Appel de la fonction  

In [4]:
identifier_enveloppe("enveloppe1",0.1, 2)

In [5]:
identifier_enveloppe("enveloppe2",0.2, 2)

In [6]:
identifier_enveloppe("enveloppe3",0.3, 2)

In [7]:
identifier_enveloppe("enveloppe4",0.1, 1)

In [8]:
identifier_enveloppe("enveloppe5",0.2, 1)

In [ ]:
identifier_enveloppe("enveloppe6",0.3, 1)